In [5]:
from pynwb import NWBHDF5IO
from franklab_nwb_extensions import fl_extension

io = NWBHDF5IO('../data/bon_04.nwb', 'r', load_namespaces=True)
nwbfile = io.read()
               
nwbfile

root pynwb.file.NWBFile at 0x4800593040
Fields:
  acquisition: {
    LFP <class 'pynwb.ecephys.LFP'>
  }
  devices: {
    NSpike acquisition system <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    001 <class 'pynwb.ecephys.ElectrodeGroup'>,
    002 <class 'pynwb.ecephys.ElectrodeGroup'>,
    003 <class 'pynwb.ecephys.ElectrodeGroup'>,
    004 <class 'pynwb.ecephys.ElectrodeGroup'>,
    005 <class 'pynwb.ecephys.ElectrodeGroup'>,
    006 <class 'pynwb.ecephys.ElectrodeGroup'>,
    007 <class 'pynwb.ecephys.ElectrodeGroup'>,
    008 <class 'pynwb.ecephys.ElectrodeGroup'>,
    010 <class 'pynwb.ecephys.ElectrodeGroup'>,
    011 <class 'pynwb.ecephys.ElectrodeGroup'>,
    012 <class 'pynwb.ecephys.ElectrodeGroup'>,
    013 <class 'pynwb.ecephys.ElectrodeGroup'>,
    014 <class 'pynwb.ecephys.ElectrodeGroup'>,
    015 <class 'pynwb.ecephys.ElectrodeGroup'>,
    017 <class 'pynwb.ecephys.ElectrodeGroup'>,
    018 <class 'pynwb.ecephys.ElectrodeGroup'>,
    019 <class 'pynwb.ecephy

In [3]:
import datajoint as dj

dj.conn()

Please enter DataJoint username:  root
Please enter DataJoint password:  ········


Connecting root@localhost:3306


DataJoint connection (connected) root@localhost:3306

In [4]:
import os

os.environ['DJ_SUPPORT_ADAPTED_TYPES'] = 'TRUE'
os.environ['DJ_SUPPORT_FILEPATH_MANAGEMENT'] = 'TRUE'
dj.config['enable_python_native_blobs'] = True

In [5]:
exported_nwb_dir = os.path.abspath('../temp_nwb/')

In [6]:
dj.config['stores'] = {
    'nwbstore': {'protocol': 'file',
                 'stage': exported_nwb_dir, # cache
                 'location': exported_nwb_dir} # same location
}

In [7]:
class NWBFileAdapter(dj.AttributeAdapter):
    attribute_type = 'filepath@nwbstore'
    
    def put(self, nwb):
        save_file_name = ''.join([nwb.identifier, '.nwb'])
        # save the file
        with NWBHDF5IO(os.path.join(exported_nwb_dir, save_file_name), mode='w') as io:
            print(f'Write NWB 2.0 file: {save_file_name}')
            io.write(nwb)
        # return the filepath to be inserted into DataJoint tables
        return os.path.join(exported_nwb_dir, save_file_name)
        
    def get(self, path):
        # read the nwb filepath and return an nwb file object back to the user
        return NWBHDF5IO(path, mode='r').read()

In [8]:
nwb_file_datatype = NWBFileAdapter()

In [9]:
schema = dj.schema('franklab')

In [10]:
@schema
class NWB(dj.Manual):
    definition = """
    nwb_id: int
    ---
    nwb: <nwb_file_datatype> 
    """

In [11]:
NWB.insert1({'nwb_id': 0, 'nwb': nwbfile})

Write NWB 2.0 file: Bon0004.nwb


ValueError: Can't change container_source once set

In [14]:
%debug

> /Users/edeno/miniconda3/envs/nwb_datajoint/lib/python3.7/site-packages/hdmf/build/manager.py(155)build()
    153                 else:
    154                     if container.container_source != source:
--> 155                         raise ValueError("Can't change container_source once set")
    156             result = self.__type_map.build(container, self, source=source, spec_ext=spec_ext)
    157             self.prebuilt(container, result)



ipdb>  container.container_source


'../data/bon_04.nwb'


ipdb>  source


*** SyntaxError: unexpected EOF while parsing


...

ipdb>  result
ipdb>  u


> /Users/edeno/miniconda3/envs/nwb_datajoint/lib/python3.7/site-packages/hdmf/utils.py(453)func_call()
    451 
    452             if is_method:
--> 453                 return func(args[0], **parsed['args'])
    454             else:
    455                 return func(**parsed['args'])



ipdb>  args


args = (<hdmf.build.manager.BuildManager object at 0xd253bd890>, root pynwb.file.NWBFile at 0x4588546512
Fields:
  acquisition: {
    LFP <class 'pynwb.ecephys.LFP'>
  }
  devices: {
    NSpike acquisition system <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    001 <class 'pynwb.ecephys.ElectrodeGroup'>,
    002 <class 'pynwb.ecephys.ElectrodeGroup'>,
    003 <class 'pynwb.ecephys.ElectrodeGroup'>,
    004 <class 'pynwb.ecephys.ElectrodeGroup'>,
    005 <class 'pynwb.ecephys.ElectrodeGroup'>,
    006 <class 'pynwb.ecephys.ElectrodeGroup'>,
    007 <class 'pynwb.ecephys.ElectrodeGroup'>,
    008 <class 'pynwb.ecephys.ElectrodeGroup'>,
    010 <class 'pynwb.ecephys.ElectrodeGroup'>,
    011 <class 'pynwb.ecephys.ElectrodeGroup'>,
    012 <class 'pynwb.ecephys.ElectrodeGroup'>,
    013 <class 'pynwb.ecephys.ElectrodeGroup'>,
    014 <class 'pynwb.ecephys.ElectrodeGroup'>,
    015 <class 'pynwb.ecephys.ElectrodeGroup'>,
    017 <class 'pynwb.ecephys.ElectrodeGroup'>,
    018 <cl

ipdb>  q
